In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import math
import pandas as pd
import arviz as az
import pickle

In [ ]:
def predict_goals_home(trace, timeframe):
    home_data = pd.read_csv(
        "/home/morten/Develop/Live-Win-Prob/data/all_games_home.csv"
    )
    R_home = np.zeros((950))
    X_home = np.zeros((9, 950))
    with pm.Model() as home_model:
        at1 = pm.Normal("at1", mu=0, sigma=math.sqrt(2))
        at2 = pm.Normal("at2", mu=0, sigma=math.sqrt(2))
        at3 = pm.Normal("at3", mu=0, sigma=math.sqrt(2))
        at4 = pm.Normal("at4", mu=0, sigma=math.sqrt(2))
        at5 = pm.Normal("at5", mu=0, sigma=math.sqrt(2))
        at6 = pm.Normal("at6", mu=0, sigma=math.sqrt(2))
        at7 = pm.Normal("at7", mu=0, sigma=math.sqrt(2))
        at8 = pm.Normal("at8", mu=0, sigma=math.sqrt(2))
        # at9 = pm.Normal("at9", mu=last_mu9, sigma=math.sqrt(2))
        # at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100,9))
        ha = pm.Normal("ha", mu=0, sigma=math.sqrt(2))
        beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

        data = pm.Data("data", X_home)
        # theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
        theta = pm.invlogit(
            (
                at1 * data[1]
                + at2 * data[2]
                + at3 * data[3]
                + at4 * data[4]
                + at5 * data[5]
                + at6 * data[6]
                + at7 * data[7]
                + at8 * data[8]
            )
            + beta
            + ha
        )
        # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)
        t = pm.Normal("t", mu=0, sigma=math.sqrt(2))
        like = pm.Poisson("like", t * theta, observed=R_home)

    X_test = np.zeros((9, 950))
    for i, row in home_data.iterrows():
        if int(i / 100) >= 950:
            for x in range(len(row)):
                if int(row[0]) == timeframe:
                    if x == 9:
                        continue
                    X_test[x][int(i / 100 - 950)] = row[x]

    with home_model:
        pm.set_data({"data": X_test}, model=home_model)
        ppc_test = pm.sample_posterior_predictive(trace, model=home_model, samples=1000)

    return ppc_test

In [ ]:
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/home_traces_100_models_verbesserung_mit_t.pkl",
    "rb",
) as f:
    home_traces = pickle.load(f)

In [ ]:
ppcs_home = []
for i in range(99):
    tmp = predict_goals_home(home_traces[i], i)
    ppcs_home.append(tmp)

In [ ]:
def predict_goals_away(trace, timeframe):
    away_data = pd.read_csv(
        "/home/morten/Develop/Live-Win-Prob/data/all_games_away.csv"
    )
    R_away = np.zeros((950))
    X_away = np.zeros((9, 950))
    with pm.Model() as away_model:
        at1 = pm.Normal("at1", mu=0, sigma=math.sqrt(2))
        at2 = pm.Normal("at2", mu=0, sigma=math.sqrt(2))
        at3 = pm.Normal("at3", mu=0, sigma=math.sqrt(2))
        at4 = pm.Normal("at4", mu=0, sigma=math.sqrt(2))
        at5 = pm.Normal("at5", mu=0, sigma=math.sqrt(2))
        at6 = pm.Normal("at6", mu=0, sigma=math.sqrt(2))
        at7 = pm.Normal("at7", mu=0, sigma=math.sqrt(2))
        at8 = pm.Normal("at8", mu=0, sigma=math.sqrt(2))
        # at9 = pm.Normal("at9", mu=last_mu9, sigma=math.sqrt(2))
        # at = pm.GaussianRandomWalk("at", mu=0, sigma=math.sqrt(2), shape=(100,9))
        beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

        data = pm.Data("data", X_away)
        # theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
        theta = pm.invlogit(
            (
                at1 * data[1]
                + at2 * data[2]
                + at3 * data[3]
                + at4 * data[4]
                + at5 * data[5]
                + at6 * data[6]
                + at7 * data[7]
                + at8 * data[8]
            )
            + beta
        )
        # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)
        t = pm.Normal("t", mu=0, sigma=math.sqrt(2))

        like = pm.Poisson("like", t * theta, observed=R_away)

    X_test = np.zeros((9, 950))
    for i, row in away_data.iterrows():
        if int(i / 100) >= 950:
            for x in range(len(row)):
                if int(row[0]) == timeframe:
                    if x == 9:
                        continue
                    X_test[x][int(i / 100 - 950)] = row[x]

    with away_model:
        pm.set_data({"data": X_test}, model=away_model)
        ppc_test = pm.sample_posterior_predictive(trace, model=away_model, samples=1000)

    return ppc_test

In [ ]:
with open(
    "/home/morten/Develop/Live-Win-Prob/dumps/away_traces_100_models_verbesserung_mit_t.pkl",
    "rb",
) as f:
    away_traces = pickle.load(f)

In [ ]:
ppcs_away = []
for i in range(100):
    tmp = predict_goals_away(away_traces[i], i)
    ppcs_away.append(tmp)

In [ ]:
away_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_away.csv")
home_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/all_games_home.csv")

In [ ]:
R_test_home = np.zeros((100, 950))
## create R
for i, row in home_data.iterrows():
    if int(i / 100) >= 950:
        R_test_home[int(row[0])][int(i / 100 - 950)] = int(row[9])
R_test_away = np.zeros((100, 950))
## create R
for i, row in away_data.iterrows():
    if int(i / 100) >= 950:
        R_test_away[int(row[0])][int(i / 100 - 950)] = int(row[9])

In [ ]:
# B_game_probs = np.zeros((53, 100, 3))
game_pred = np.zeros((100, 53))
acc_pred = np.zeros((100, 53))

In [ ]:
for g in range(53):
    game_num = g
    game_probs = np.zeros((100, 3))
    goal_home_idx = []
    goal_away_idx = []
    for i in range(100):
        hist_home = np.histogram(
            ppcs_home[i]["like"][:, game_num],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        hist_away = np.histogram(
            ppcs_away[i]["like"][:, game_num],
            bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            density=True,
        )
        goals_home = R_test_home[0][game_num] - R_test_home[i][game_num]
        goals_away = R_test_away[0][game_num] - R_test_away[i][game_num]
        if i > 0:
            if R_test_home[i][game_num] != R_test_home[i - 1][game_num]:
                goal_home_idx.append(i)
            if R_test_away[i][game_num] != R_test_away[i - 1][game_num]:
                goal_away_idx.append(i)
        goal_matrix = np.zeros((9, 9))
        home_prob = 0
        away_prob = 0
        draw_prob = 0
        for h in range(9):
            for a in range(9):
                goal_matrix[h][a] = hist_home[0][h] * hist_away[0][a]
                home = goals_home + h
                away = goals_away + a
                if home > away:
                    home_prob += hist_home[0][h] * hist_away[0][a]
                elif away > home:
                    away_prob += hist_home[0][h] * hist_away[0][a]
                elif home == away:
                    draw_prob += hist_home[0][h] * hist_away[0][a]

        if home_prob >= draw_prob and home_prob >= away_prob:
            game_pred[i][g] = 0
            acc_pred[i][g] = home_prob
        elif away_prob >= draw_prob and away_prob >= home_prob:
            game_pred[i][g] = 2
            acc_pred[i][g] = away_prob
        elif draw_prob >= away_prob and draw_prob >= home_prob:
            game_pred[i][g] = 1
            acc_pred[i][g] = draw_prob

In [ ]:
act_res = np.zeros((53))
for g in range(53):
    goals_diff = R_test_home[0][g] - R_test_away[0][g]
    if goals_diff < 0:
        act_res[g] = 2
    elif goals_diff == 0:
        act_res[g] = 1

In [ ]:
def acc_ece(pred, label):
    pred = np.array(pred)
    mul = 1 / pred.size
    sum_ece = 0
    for min in range(len(pred)):
        for game in range(len(pred[min])):
            if pred[min][game] == label[game]:
                sum_ece += 1
    return mul * sum_ece

In [ ]:
def conf_ece(prob):
    prob = np.array(prob)
    mul = 1 / prob.size
    mul2 = np.sum(prob)
    return mul * mul2

In [ ]:
def calc_ece(score_bins, prob_bins, labels):
    M = len(score_bins)
    N = 0
    ece = 0
    for m in range(M):
        N += len(score_bins[m])
    for m in range(M):
        ece += (len(score_bins[m]) / N) * np.abs(
            acc_ece(score_bins[m], labels) - conf_ece(prob_bins[m])
        )

    return ece

In [ ]:
prob_pred_bins, score_pred_bins = [], []
prob_pred_bins.append(acc_pred[0:10])
prob_pred_bins.append(acc_pred[10:20])
prob_pred_bins.append(acc_pred[20:30])
prob_pred_bins.append(acc_pred[30:40])
prob_pred_bins.append(acc_pred[40:50])
score_pred_bins.append(game_pred[0:10])
score_pred_bins.append(game_pred[10:20])
score_pred_bins.append(game_pred[20:30])
score_pred_bins.append(game_pred[30:40])
score_pred_bins.append(game_pred[40:50])

ece = calc_ece(score_pred_bins, prob_pred_bins, act_res)
print(ece)

In [ ]:
prob_pred_bins, score_pred_bins = [], []
prob_pred_bins.append(acc_pred[50:60])
prob_pred_bins.append(acc_pred[60:70])
prob_pred_bins.append(acc_pred[70:80])
prob_pred_bins.append(acc_pred[80:90])
prob_pred_bins.append(acc_pred[90:100])
score_pred_bins.append(game_pred[50:60])
score_pred_bins.append(game_pred[60:70])
score_pred_bins.append(game_pred[70:80])
score_pred_bins.append(game_pred[80:90])
score_pred_bins.append(game_pred[90:100])

ece = calc_ece(score_pred_bins, prob_pred_bins, act_res)
print(ece)

In [ ]:
prob_pred_bins, score_pred_bins = [], []
prob_pred_bins.append(acc_pred[90:92])
prob_pred_bins.append(acc_pred[92:94])
prob_pred_bins.append(acc_pred[94:96])
prob_pred_bins.append(acc_pred[96:98])
prob_pred_bins.append(acc_pred[98:100])
score_pred_bins.append(game_pred[90:92])
score_pred_bins.append(game_pred[92:94])
score_pred_bins.append(game_pred[94:96])
score_pred_bins.append(game_pred[96:98])
score_pred_bins.append(game_pred[98:100])

ece = calc_ece(score_pred_bins, prob_pred_bins, act_res)
print(ece)

In [ ]:
prob_pred_bins, score_pred_bins = [], []
prob_pred_bins.append(acc_pred[0:20])
prob_pred_bins.append(acc_pred[20:40])
prob_pred_bins.append(acc_pred[40:60])
prob_pred_bins.append(acc_pred[60:80])
prob_pred_bins.append(acc_pred[80:100])
score_pred_bins.append(game_pred[0:20])
score_pred_bins.append(game_pred[20:40])
score_pred_bins.append(game_pred[40:60])
score_pred_bins.append(game_pred[60:80])
score_pred_bins.append(game_pred[80:100])

ece = calc_ece(score_pred_bins, prob_pred_bins, act_res)
print(ece)

In [ ]:
game_num = 17

hist_home, bins_h = np.histogram(
    ppcs_home[28]["like"][:, game_num],
    bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    density=True,
)
hist_away, bins_a = np.histogram(
    ppcs_away[28]["like"][:, game_num],
    bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    density=True,
)

width = 0.7 * (bins_h[1] - bins_h[0])
center = (bins_h[:-1] + bins_h[1:]) / 2
plt.bar(center, hist_home, align="center", width=width)
plt.show()

In [ ]:
game_num = 17

hist_home, bins_h = np.histogram(
    ppcs_home[28]["like"][:, game_num],
    bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    density=True,
)
hist_away, bins_a = np.histogram(
    ppcs_away[28]["like"][:, game_num],
    bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    density=True,
)

width = 0.7 * (bins_h[1] - bins_a[0])
center = (bins_a[:-1] + bins_a[1:]) / 2
plt.bar(center, hist_away, align="center", width=width)
plt.show()

In [ ]:
score_matrix = np.zeros((6, 6))
for h in range(6):
    for a in range(6):
        score_matrix[a][h] = (hist_home[h] * hist_away[a]) * 100

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.heatmap(score_matrix, annot=True, fmt=".0f", cbar=False, cmap="rocket_r")
ax.xaxis.tick_top()
ax.set_xlabel("Home")
ax.xaxis.set_label_position("top")
ax.set_ylabel("Away")
ax.margins(x=0, y=0)


fig.show()